In [1]:
# ------------------------------------------------------
# a. Import Required Libraries
# ------------------------------------------------------
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import SGD

# ------------------------------------------------------
# b. Upload / Access the Dataset
# ------------------------------------------------------
data = pd.read_csv("C:/Users/UMESH/OneDrive/Desktop/creditcard.csv")

# Separate features and labels
X = data.drop("Class", axis=1).values
y = data["Class"].values

# Normalize
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# ------------------------------------------------------
# c. Encoder: Convert Input to Latent Space
# ------------------------------------------------------
input_dim = X_scaled.shape[1]
latent_dim = 4

input_layer = Input(shape=(input_dim,))
encoder_layer1 = Dense(8, activation='relu')(input_layer)
encoder_output = Dense(latent_dim, activation='relu')(encoder_layer1)

# ------------------------------------------------------
# d. Decoder: Reconstruct the Input
# ------------------------------------------------------
decoder_layer1 = Dense(8, activation='relu')(encoder_output)
decoder_output = Dense(input_dim, activation='sigmoid')(decoder_layer1)

# Full Autoencoder model
autoencoder = Model(inputs=input_layer, outputs=decoder_output)

# ------------------------------------------------------
# e. Compile the Model (Optimizer, Loss, Metrics)
# ------------------------------------------------------
autoencoder.compile(
    optimizer=SGD(learning_rate=0.01),
    loss='mse',
    metrics=['mae']
)

autoencoder.summary()

# ------------------------------------------------------
# Train Only on Normal Data (Class 0)
# ------------------------------------------------------
X_normal = X_scaled[y == 0]

history = autoencoder.fit(
    X_normal, X_normal,
    epochs=5,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

# ------------------------------------------------------
# Anomaly Detection
# ------------------------------------------------------

# Reconstruct entire dataset
reconstructed = autoencoder.predict(X_scaled)

# Reconstruction Error
mse = np.mean(np.square(X_scaled - reconstructed), axis=1)

# Threshold (Top 95 percentile)
threshold = np.percentile(mse, 95)
print("Threshold:", threshold)

# Predict anomalies
y_pred = (mse > threshold).astype(int)

# ------------------------------------------------------
# Evaluation
# ------------------------------------------------------
print("\nClassification Report:")
print(classification_report(y, y_pred))




Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30)]              0         
                                                                 
 dense (Dense)               (None, 8)                 248       
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 8)                 40        
                                                                 
 dense_3 (Dense)             (None, 30)                270       
                                                                 
Total params: 594 (2.32 KB)
Trainable params: 594 (2.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5


7997/7997 [======

In [ ]:
# ------------------------------------------------------
# Anomaly Check for Each Record (One-by-One)
# ------------------------------------------------------

print("\nChecking anomaly for every record...\n")

record_errors = []
record_preds = []

for i in range(len(X_scaled)):
    # Take one record
    one_record = X_scaled[i].reshape(1, -1)
    
    # Reconstruct
    one_reconstructed = autoencoder.predict(one_record, verbose=0)
    
    # Compute reconstruction error
    one_mse = np.mean((one_record - one_reconstructed)**2)
    record_errors.append(one_mse)
    
    # Predict anomaly
    one_pred = 1 if one_mse > threshold else 0
    record_preds.append(one_pred)
    
    # Optional: print for each record
    print(f"Record {i}:  Error = {one_mse:.6f}  ->  Anomaly = {one_pred}")

print("\nFinished checking all records.")



Checking anomaly for every record...

Record 0:  Error = 0.009428  ->  Anomaly = 1
Record 1:  Error = 0.009283  ->  Anomaly = 1
Record 2:  Error = 0.011502  ->  Anomaly = 1
Record 3:  Error = 0.010684  ->  Anomaly = 1
Record 4:  Error = 0.009532  ->  Anomaly = 1
Record 5:  Error = 0.008993  ->  Anomaly = 0
Record 6:  Error = 0.009622  ->  Anomaly = 1
Record 7:  Error = 0.009985  ->  Anomaly = 1
Record 8:  Error = 0.009543  ->  Anomaly = 1
Record 9:  Error = 0.009184  ->  Anomaly = 1
Record 10:  Error = 0.009488  ->  Anomaly = 1
Record 11:  Error = 0.009872  ->  Anomaly = 1
Record 12:  Error = 0.009791  ->  Anomaly = 1
Record 13:  Error = 0.009663  ->  Anomaly = 1
Record 14:  Error = 0.010245  ->  Anomaly = 1
Record 15:  Error = 0.009597  ->  Anomaly = 1
Record 16:  Error = 0.009240  ->  Anomaly = 1
Record 17:  Error = 0.009280  ->  Anomaly = 1
Record 18:  Error = 0.009833  ->  Anomaly = 1
Record 19:  Error = 0.009321  ->  Anomaly = 1
Record 20:  Error = 0.009813  ->  Anomaly = 1
Recor